In [13]:
%reset -f
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate as si
import chameleon as c

I want to use the scipy integrate method `solve_ivp` to solve a system of differential equations.

In [14]:
m = 6
L = 5
cd = 2
a = 3
E = 169 / 120
T = 10

In [15]:
def sig_a(env, t):
    return -(np.sin(env.pos_init) ** 2) * np.sin(t) ** 2


def integrate_to_t(env, t):
    """Integrate the environment from 0 to t"""
    time_steps = int(t / env.dt)
    t_arr = np.linspace(0, t, time_steps)
    for t in t_arr:
        sig = sig_a(env, t)
        env.one_step_return(sig)
    return


env = c.Chameleon(dt=0.01, c=cd, m=m, init_length=L, E=E, alpha=a)
integrate_to_t(env, T)

CPU times: user 966 ms, sys: 8.41 ms, total: 974 ms
Wall time: 982 ms


In [ ]:
def G(t, L):
    env = c.Chameleon(init_length=L)
    sig = sig_a(env, t)
    dx = env.pos_init[1] - env.pos_init[0]
    g = -si.trapezoid(sig, dx=dx)
    return g


def system(t, y, m, L, c, a, E, G_fun):
    """simulate dy/dt = -y"""
    mat = np.array([[0, 1], [-E / (L * m), -(L * c + a) / (L * m)]])
    y_out = np.matmul(mat, y) + np.array([0, G_fun(t, L) / (L * m)])
    return y_out


sol = si.solve_ivp(
    system,
    t_span=[0, T],
    method="BDF",
    dense_output=True,
    y0=np.array([0, 0]),
    args=[m, L, cd, a, E, G],
)

In [ ]:
y1 = sol["y"][0]
y2 = sol["y"][1]
t = sol["t"]
F = -cd * y2 - m * np.gradient(y2, t, edge_order=2)

In [ ]:
t.shape

In [ ]:
analytic = [
    {0.0},
    {-0.0319976},
    {-0.119079},
    {-0.238296},
    {-0.358829},
    {-0.449538},
    {-0.48659},
    {-0.459301},
    {-0.37275},
    {-0.246546},
    {-0.110022},
    {0.00493849},
    {0.0717093},
    {0.0754366},
    {0.016675},
    {-0.0887497},
    {-0.213616},
    {-0.325974},
    {-0.396965},
    {-0.407891},
    {-0.354794},
    {-0.249419},
    {-0.116348},
    {0.0130285},
    {0.108188},
    {0.146956},
    {0.120931},
    {0.0375431},
    {-0.0817636},
    {-0.206783},
    {-0.30594},
    {-0.354024},
    {-0.338355},
    {-0.261895},
    {-0.142516},
    {-0.00862621},
    {0.107784},
    {0.17898},
    {0.188268},
    {0.134088},
    {0.0303937},
]
analytic = [s.pop() for s in analytic]
trange = np.linspace(0, T, len(analytic))

In [ ]:
plt.figure(figsize=(10, 10))
plt.plot(t, F)
plt.plot(np.linspace(0, T, len(env.F)), env.F)
plt.plot(trange, analytic)
plt.xlabel("t")
plt.ylabel("$u_L(t)$")
plt.legend((r"$F(t)$", "F from sim", "F analytic"))
plt.show()